# SYCL with USM to RAJA-SYCL

## Starting Point
Now that we have an `nd_range` with USM implementation in SYCL, we can write the corresponding code in RAJA. The RAJA-SYCL backend that we will use is a work in progress and not yet upstream in RAJA.  The working draft can be found at https://github.com/homerdin/RAJA.

```c
#include "DAXPY.hpp"

#include "RAJA/RAJA.hpp"

#if defined(RAJA_ENABLE_SYCL)

#include "common/SyclDataUtils.hpp"

#include <iostream>

namespace rajaperf
{
namespace basic
{

  //
  // Define thread block size for SYCL execution
  //
  const size_t block_size = 256;

#define DAXPY_DATA_SETUP_SYCL \
  allocAndInitSyclDeviceData(x, m_x, iend, qu); \
  allocAndInitSyclDeviceData(y, m_y, iend, qu);

#define DAXPY_DATA_TEARDOWN_SYCL \
  getSyclDeviceData(m_y, y, iend, qu); \
  deallocSyclDeviceData(x, qu); \
  deallocSyclDeviceData(y, qu);


void DAXPY::runSyclVariant(VariantID vid)
{
  const Index_type run_reps = getRunReps();
  const Index_type ibegin = 0;
  const Index_type iend = getRunSize();

  DAXPY_DATA_SETUP;

  if ( vid == Base_SYCL ) {

    DAXPY_DATA_SETUP_SYCL;

    startTimer();
    for (RepIndex_type irep = 0; irep < run_reps; ++irep) {

      const size_t global_size = block_size * RAJA_DIVIDE_CEILING_INT(iend, block_size);

      qu.submit([&] (cl::sycl::handler& h) {
        h.parallel_for<class DAXPY>(cl::sycl::nd_range<1>{global_size, block_size},
                                    [=] (cl::sycl::nd_item<1> item ) {

          Index_type i = item.get_global_id(0);
          if (i < iend) {
            DAXPY_BODY
          }

        });
      });
    }
    qu.wait(); // Wait for computation to finish before stopping timer
    stopTimer();

    DAXPY_DATA_TEARDOWN_SYCL;

  } else if ( vid == RAJA_SYCL ) {

      // Lets fill this in
      
  } else {
     std::cout << "\n  DAXPY : Unknown Sycl variant id = " << vid << std::endl;
  }

}

} // end namespace basic
} // end namespace rajaperf

#endif  // RAJA_ENABLE_SYCL
```

## Data Setup
We will continue to use the USM implementations for the memory management, so no changes are needed.  

RAJA is an abstraction for portable, parallel loop execution.  RAJA does not provide any abstractions over memory management.  There is a related project UMPIRE which abstracts away programming model specific memory management.

## Kernel Launch
The RAJA kernel launch look similar to the sycl kernel launch.  However we do not need to calculate the global size and guard the execution of the body, this is handled by RAJA.  Instead we create a `RAJA::forall` loop, which will use the `sycl_exec` execution policy.  This tells RAJA to use the SYCL backend.  The template parameters, `<block_size, true>`, specify the block size to use and whether to launch the loop asynchronously.  
```c
      RAJA::forall< RAJA::sycl_exec<block_size, true> >(
        RAJA::RangeSegment(ibegin, iend), [=] (Index_type i) {
        DAXPY_BODY;
      });
```

When coming from an existing RAJA implementation all that is needed is to changes the execution policy to the one you are now targeting (OpenMPTarget or CUDA below).  So everything would be the same except we need to change :
```c
RAJA::sycl_exec<block_size, true>
```
to 
```c
RAJA::omp_target_parallel_for_exec<threads_per_team>
``` 
or 
```c
RAJA::cuda_exec<block_size, true>
```

## Kernel
RAJA maps our iteration location to the given index.  This is straightforward for a basic `forall`, but with more complex kernel structures we can easily change how to map our index variables to the kernel iteration locations.
```c
      [=] (Index_type i) {
        DAXPY_BODY;
      });
```

## Data Teardown
Again we will use the existing USM implementation for memory management.

## Lets put it all together

```c
//~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~//
// Copyright (c) 2017-19, Lawrence Livermore National Security, LLC.
//
// Produced at the Lawrence Livermore National Laboratory
//
// LLNL-CODE-738930
//
// All rights reserved.
//
// This file is part of the RAJA Performance Suite.
//
// For details about use and distribution, please read RAJAPerf/LICENSE.
//
//~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~//

#include "DAXPY.hpp"

#include "RAJA/RAJA.hpp"

#if defined(RAJA_ENABLE_SYCL)

#include "common/SyclDataUtils.hpp"

#include <iostream>

namespace rajaperf
{
namespace basic
{

  //
  // Define thread block size for SYCL execution
  //
  const size_t block_size = 256;

#define DAXPY_DATA_SETUP_SYCL \
  allocAndInitSyclDeviceData(x, m_x, iend, qu); \
  allocAndInitSyclDeviceData(y, m_y, iend, qu);

#define DAXPY_DATA_TEARDOWN_SYCL \
  getSyclDeviceData(m_y, y, iend, qu); \
  deallocSyclDeviceData(x, qu); \
  deallocSyclDeviceData(y, qu);


void DAXPY::runSyclVariant(VariantID vid)
{
  const Index_type run_reps = getRunReps();
  const Index_type ibegin = 0;
  const Index_type iend = getRunSize();

  DAXPY_DATA_SETUP;

  if ( vid == Base_SYCL ) {

    // We already did this

  } else if ( vid == RAJA_SYCL ) {

    DAXPY_DATA_SETUP_SYCL;
    startTimer();
    for (RepIndex_type irep = 0; irep < run_reps; ++irep) {

      RAJA::forall< RAJA::sycl_exec<block_size, true> >(
        RAJA::RangeSegment(ibegin, iend), [=] (Index_type i) {
        DAXPY_BODY;
      });

    }
    qu.wait();
    stopTimer();

    DAXPY_DATA_TEARDOWN_SYCL;

  } else {
     std::cout << "\n  DAXPY : Unknown Sycl variant id = " << vid << std::endl;
  }

}

} // end namespace basic
} // end namespace rajaperf

#endif  // RAJA_ENABLE_SYCL
```

In [ ]:
# Now Lets Run It;